In [1]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
import datetime

In [2]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [3]:
username = 'vigrose'
password_edw = 'Slytherin~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [ ]:
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [4]:
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [ ]:
#MSA_ZR
LAT_QUERY_2 = \
    f"""
    SELECT 
    Z.ZIP,
    M.MSA_CD,
    M.POPULATION,
    M.MSA_TYPE,
    Z.CBSA,
    C.LEVEL_MSA,
    C.STATUS_1_OR_2
    FROM
    AMAEDW.CBSA_ZR C, AMAEDW.ZIP_CBSA_ZR Z, AMAEDW.MSA_ZR M
    WHERE
    C.ZIP_CBSA_DIVISION = Z.CBSA
    AND
    Z.ACTIVE_IND = 'Y'
    AND
    C.ACTIVE_IND = 'Y'
    AND
    C.CBSA_OR_DIV = 'CBSA'
    AND
    M.ACTIVE_IND = 'Y'
    """
hmm = pd.read_sql(con=AMAEDW, sql=LAT_QUERY_2)

In [ ]:
FAX_QUERY = \
    """
    SELECT 
    P.PHONE_ID,
    P.PARTY_ID,
    N.AREA_CD,
    N.EXCHANGE,
    N.PHONE_NBR,
    P.FROM_DT,
    P.THRU_DT,
    P.CAT_CD_ID
    FROM
    AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.PARTY_PHONE P, AMAEDW.PHONE_NBR N
    WHERE
    C.PURPOSE_TYPE_ID = P.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD = 'F'
    AND
    P.PHONE_ID = N.PHONE_ID
    AND
    P.THRU_DT is null
    """
fax = pd.read_sql(con=AMAEDW, sql=FAX_QUERY)

In [13]:
PHONE_QUERY = \
    """
    SELECT 
    P.PHONE_ID,
    P.PARTY_ID,
    N.AREA_CD,
    N.EXCHANGE,
    N.PHONE_NBR,
    P.FROM_DT,
    P.THRU_DT,
    P.CAT_CD_ID,
    C.PURPOSE_USG_CD
    FROM
    AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.PARTY_PHONE P, AMAEDW.PHONE_NBR N
    WHERE
    C.PURPOSE_TYPE_ID = P.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD = 'P'
    AND
    P.PHONE_ID = N.PHONE_ID
    AND
    P.THRU_DT is null
    """
phone = pd.read_sql(con=AMAEDW, sql=PHONE_QUERY)

In [ ]:
EMAIL_QUERY = \
    """
    SELECT 
    P.EMAIL_ID,
    P.PARTY_ID,
    E.EMAIL_ADDR,
    E.EMAIL_STATUS,
    P.FROM_DT,
    P.THRU_DT,
    P.CAT_CD_ID,
    C.PURPOSE_USG_CD
    FROM
    AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.PARTY_EMAIL P, AMAEDW.EMAIL_ADDR E
    WHERE
    C.PURPOSE_TYPE_ID = P.PURPOSE_TYPE_ID
    AND
    P.EMAIL_ID = E.EMAIL_ID
    AND
    P.THRU_DT is null
    """
email = pd.read_sql(con=AMAEDW, sql=EMAIL_QUERY)

In [15]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [6]:
POSTCD_QUERY = \
    """
    SELECT
    P.POST_CD_ID, 
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    P.STATE_ID,
    P.POST_CD,
    P.POST_CD_PLUS_4,
    P.LATITUDE,
    P.LONGITUDE
    FROM
    AMAEDW.POST_CD P
    """
post_cd = pd.read_sql(con=AMAEDW, sql=POSTCD_QUERY)

In [7]:
PARTY_ADDR_QUERY = \
    """
    SELECT 
    A.POST_CD_ID,
    A.PARTY_ID,
    A.FROM_DT,
    A.THRU_DT,
    A.CAT_CD_ID,
    C.PURPOSE_USG_CD,
    A.PARTY_ADDR_ID,
    C.PURPOSE_TYPE_CD,
    C.PURPOSE_USG_DESC
    FROM
    AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.PARTY_ADDR A
    WHERE
    C.PURPOSE_TYPE_ID = A.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD = 'A'
    AND
    A.THRU_DT is null
    AND
    C.SRC_SYS = 'MASTERFILE'
    """
party_addr = pd.read_sql(con=AMAEDW, sql=PARTY_ADDR_QUERY)

In [19]:
# ids = pd.merge(party_ids, ov_me, left_on='ME', right_on='medical_education_number')
# polos = pd.merge(party_addr[party_addr.PURPOSE_USG_CD=='PO  '], post_cd, on='POST_CD_ID')
# polo_stuff = pd.merge(ids, polos, on='PARTY_ID')
polo_stuff = pd.merge(polo_stuff, phone, on='PARTY_ID')

In [20]:
codes = pd.read_csv('../../Data/Measurement/PhoneAreaCode.csv')

In [43]:
state_tbl = pd.read_csv('../../Data/Measurement/STATE.csv')
state_tbl.head()

,SRC_STATE_CD,STATE_ID,DESC,SRC_SYS,INSERT_TMSTP,UPDATE_TMSTP,INSERT_USERID,UPDATE_USERID,AMA_STATE_CD
0,~,-1,NOT AVAILABLE / UNKNOWN,MANUAL,13AUG2015:00:00:00.000000,13AUG2015:00:00:00.000000,MANUAL,MANUAL,NaN
1,AK,195,Alaska,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,2.0
2,AL,196,Alabama,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,1.0
3,CT,197,Connecticut,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,8.0
4,FL,198,Florida,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,11.0


In [32]:
polo_stuff.AREA_CD = [int(x) for x in polo_stuff.AREA_CD]

In [33]:
polo_stuff = pd.merge(polo_stuff, codes, right_on='area_code', left_on='AREA_CD')

In [44]:
polo_stuff = pd.merge(polo_stuff, state_tbl[['SRC_STATE_CD', 'STATE_ID', 'DESC']], on='STATE_ID')

In [62]:
polo_stuff[polo_stuff.DESC!=polo_stuff.state][['ME','CITY','DESC','state','city','PURPOSE_USG_CD_y']]

,ME,CITY,DESC,state,city,PURPOSE_USG_CD_y
5918,00102952138,Phenix City,Alabama,Georgia,Augusta,None
5919,00102952138,Phenix City,Alabama,Georgia,Augusta,PV
5920,01222000358,Montgomery,Alabama,Georgia,Augusta,None
5921,01222000358,Montgomery,Alabama,Georgia,Augusta,PV
5922,01222000358,Montgomery,Alabama,Georgia,Augusta,AMC
...,...,...,...,...,...,...
2389487,03306060987,DPO,Federal Services,Virginia,Roanoke,PV
2389488,03841031201,APO,Federal Services,Louisiana,Lafayette,None
2389489,03841031201,APO,Federal Services,Louisiana,Lafayette,PV
2389490,02101060449,DPO,Federal Services,New York,New York,None


In [60]:
len(polo_stuff)

2389497

In [61]:
1978507/2389497

0.8280014580474468

In [54]:
polo_stuff.CITY = [x.strip() for x in polo_stuff.CITY]

In [67]:
polo_stuff.PURPOSE_USG_CD_y.unique()

array(['None', 'PV  ', 'WEB ', 'CVNT', 'AMC ', 'MSOP', 'MBR '],
      dtype=object)

In [84]:
GOOD_NOT_PV = polo_stuff[(polo_stuff.PURPOSE_USG_CD_y!='PV  ')&(polo_stuff.DESC==polo_stuff.state)].ME

In [75]:
BAD_PV = polo_stuff[(polo_stuff.PURPOSE_USG_CD_y=='PV  ')&(polo_stuff.DESC!=polo_stuff.state)].ME

In [82]:
polo_stuff[(polo_stuff.ME.isin(GOOD_NOT_PV))&polo_stuff.ME.isin(BAD_PV)][['ME','FROM_DT_y','FROM_DT_x','CITY','DESC','state','city','PURPOSE_USG_CD_y','AREA_CD','CAT_CD_ID_y']].sort_values('ME').to_csv('../../Data/Measurement/better_phones.csv', index=False)

In [88]:
polo_stuff[(polo_stuff.ME.isin(GOOD_NOT_PV))&polo_stuff.ME.isin(BAD_PV)][['ME','FROM_DT_y','FROM_DT_x','CITY','DESC','state','city','PURPOSE_USG_CD_y','AREA_CD','CAT_CD_ID_y']].sort_values('ME').to_csv('../../Data/Measurement/other_better_phones.csv', index=False)

In [97]:
polo_stuff['TELEPHONE'] = [str(x) for x in polo_stuff.AREA_CD]+polo_stuff.EXCHANGE+polo_stuff.PHONE_NBR

In [102]:
phone_addr = polo_stuff[(polo_stuff.PURPOSE_USG_CD_y=='PV  ')].groupby(['POST_CD_ID','TELEPHONE']).count()[['PARTY_ID']]

In [105]:
phon_addr = phone_addr[phone_addr.PARTY_ID>2]

In [110]:
# phone_addr.to_csv('../../Data/Measurement/phone_addr.csv')
phone_addr = pd.read_csv('../../Data/Measurement/phone_addr.csv')

In [112]:
PV_POLO = polo_stuff[(polo_stuff.PURPOSE_USG_CD_y=='PV  ')]

In [113]:
PV_POLO[(PV_POLO.POST_CD_ID.isin(phone_addr.POST_CD_ID))&~(PV_POLO.TELEPHONE.isin(phone_addr.TELEPHONE))]

,PARTY_ID,ME,medical_education_number,type,POST_CD_ID,FROM_DT_x,THRU_DT_x,CAT_CD_ID_x,PURPOSE_USG_CD_x,PARTY_ADDR_ID,...,FROM_DT_y,THRU_DT_y,CAT_CD_ID_y,PURPOSE_USG_CD_y,area_code,state,city,SRC_STATE_CD,DESC,TELEPHONE
2,1892959,00106930210,00106930210,Physician,16561405,2005-12-16 20:35:36.375262,None,NaN,PO,0,...,2012-10-17 23:05:55.175551,None,3923.0,PV,256,Alabama,Huntsville,AL,Alabama,2563061655
9,1904336,00102801806,00102801806,Physician,16270455,2005-12-16 20:23:19.418674,None,NaN,PO,0,...,2005-12-16 20:24:34.138153,None,3952.0,PV,256,Alabama,Huntsville,AL,Alabama,2565432273
16,1892070,00102811356,00102811356,Physician,16441353,2005-12-16 20:29:06.402006,None,NaN,PO,0,...,2005-12-16 20:24:37.620210,None,3952.0,PV,256,Alabama,Huntsville,AL,Alabama,2564427683
23,1892079,00102820347,00102820347,Physician,32558060,2021-09-10 20:46:34.767046,None,4032.0,PO,-1,...,2009-06-18 21:20:43.769399,None,3819.0,PV,256,Alabama,Huntsville,AL,Alabama,2563830423
26,1903930,00102710967,00102710967,Physician,32393479,2021-07-09 20:59:16.955076,None,4032.0,PO,-1,...,2005-12-16 20:30:25.083178,None,3819.0,PV,256,Alabama,Huntsville,AL,Alabama,2565337751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2389420,2940712,01401030286,01401030286,Physician,31153026,2022-07-08 07:39:46.875765,None,4159.0,PO,-1,...,2019-04-02 21:54:57.319713,None,6611.0,PV,671,Guam,NaN,GU,Guam,6717353339
2389426,2964626,74811890298,74811890298,Physician,35119136,2022-07-22 07:17:59.599249,None,6455.0,PO,-1,...,2022-04-29 20:45:48.833168,None,6790.0,PV,671,Guam,NaN,GU,Guam,6716468845
2389447,3321595,74810950190,74810950190,Physician,34687160,2022-02-17 20:51:26.612561,None,6772.0,PO,-1,...,2021-10-21 20:44:25.053950,None,6775.0,PV,671,Guam,NaN,GU,Guam,6716468765
2389485,1888367,01401880751,01401880751,Physician,35195340,2022-08-14 20:47:53.998959,None,6672.0,PO,-1,...,2022-08-14 20:47:54.538515,None,6672.0,PV,684,American Samoa,NaN,AS,American Samoa,6846335871


In [ ]:
#address undeliverable
#AIMS.ENTITY_COMM_EXC_CT
UNDELIVERABLE_QUERY = \
    f'''
    SELECT
    ENTITY_ID,
    COMM_ID,
    COMM_EXC_CAT_CODE
    FROM
    informix.ENTITY_COMM_EXC_CT
    WHERE
    END_DT IS NULL
    ;
    '''
undeliverable = pd.read_sql(con=informix, sql=UNDELIVERABLE_QUERY)

In [ ]:
#EDW.STATE_DIV_REG_LKP
STATE_DIV_QUERY = \
    """
    SELECT *
    FROM
    AMAEDW.STATE_DIV_REG_LKP
    """
census_div = pd.read_sql(con=AMAEDW, sql=STATE_DIV_QUERY)

In [ ]:
ids = pd.merge(party_ids, ov_me, left_on='ME', right_on='medical_education_number')

In [ ]:
# polos = party_addr[party_addr.PURPOSE_USG_CD=='PO  '][['PARTY_ID', 'POST_CD_ID']].drop_duplicates('PARTY_ID').rename(columns ={'POST_CD_ID':'POLO_ADDRESS'})
# ppmas = party_addr[party_addr.PURPOSE_USG_CD=='PP  '][['PARTY_ID', 'POST_CD_ID']].drop_duplicates('PARTY_ID').rename(columns ={'POST_CD_ID':'PPMA_ADDRESS'})
# addresses = party_addr[['PARTY_ID', 'POST_CD_ID']].drop_duplicates('PARTY_ID').rename(columns ={'POST_CD_ID':'ADDRESS'})
# preferred_phones = phone[phone.PURPOSE_USG_CD=='PV  '][['PARTY_ID', 'PHONE_ID']].drop_duplicates('PARTY_ID').rename(columns ={'PHONE_ID':'PREFERRED_PHONE'})
# phones = phone[['PARTY_ID', 'PHONE_ID']].drop_duplicates('PARTY_ID')
faxes = fax[['PARTY_ID', 'PHONE_ID']].drop_duplicates('PARTY_ID').rename(columns ={'PHONE_ID':'FAX_ID'})
preferred_emails = email[email.PURPOSE_USG_CD=='PE  '][['PARTY_ID', 'EMAIL_ID']].drop_duplicates('PARTY_ID').rename(columns ={'EMAIL_ID':'PREFERRED_EMAIL'})
emails = email[['PARTY_ID', 'EMAIL_ID']].drop_duplicates('PARTY_ID')

In [ ]:
PARTY_LEVEL = pd.merge(ids, addresses, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, ppmas, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, polos, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, phones, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, preferred_phones, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, emails, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, preferred_emails, on='PARTY_ID', how='left')
PARTY_LEVEL = pd.merge(PARTY_LEVEL, faxes, on='PARTY_ID', how='left')

In [ ]:
party_addr

In [ ]:
#party level
today = datetime.date.today()
types = ['Physician','Student','Resident']
all_mes = list(PARTY_LEVEL.ME)
elements = [
    {'data_element': 'Address',
     'element':'ADDRESS',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'PPMA Address',
     'element':'PPMA_ADDRESS',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'POLO Address',
     'element':'POLO_ADDRESS',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'Phone',
     'element':'PHONE_ID',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'Preferred Phone',
     'element':'PREFERRED_PHONE',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'Fax',
     'element':'FAX_ID',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'Email',
     'element':'EMAIL_ID',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'Preferred Email',
     'element':'PREFERRED_EMAIL',
    'null': [''],
    'universe': all_mes}
]
dict_list = []
for typo in types:
    total = len(PARTY_LEVEL[PARTY_LEVEL.type==typo])
    for element in elements:
        universe = len(PARTY_LEVEL[(PARTY_LEVEL.type==typo)&(PARTY_LEVEL.ME.isin(element['universe']))])
        complete = len(PARTY_LEVEL[(PARTY_LEVEL.type==typo)&~(PARTY_LEVEL[element['element']].isna())&(PARTY_LEVEL.ME.isin(element['universe']))])
        true_complete = len(PARTY_LEVEL[(PARTY_LEVEL.type==typo)&~(PARTY_LEVEL[element['element']].isin(element['null']))&~(PARTY_LEVEL[element['element']].isna())&(PARTY_LEVEL.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness'
        }
        dict_list.append(new_dict)

In [ ]:
bop = pd.DataFrame(dict_list)
bop['Completeness'] = bop['Complete']/bop['Universe Total']*100

In [ ]:
bop

In [ ]:
today = datetime.date.today()
types = ['Physician','Student','Resident']
all_mes = list(ALL.ME)
us_schools = list(ALL[(ALL.COUNTRY_ID_SCH==6705)].ME)
us_gmes = list(ALL[(ALL.COUNTRY_ID_GME==6705)].ME)
elements = [
    {'data_element': 'Medical School Name',
     'element':'ORG_NM_SCH',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'Medical School State',
     'element':'STATE_ID_SCH',
    'null': ['',-1],
    'universe': us_schools},
    {'data_element': 'Medical School Country',
     'element':'COUNTRY_ID_SCH',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'Medical School Grad Date',
     'element':'GRAD_DT',
    'null': ['',-1, datetime.date(1900, 1, 30)],
    'universe': all_mes},
    {'data_element': 'Education Status',
     'element':'STS_TYPE_ID',
    'null': ['',-1,61,43],
    'universe': all_mes},
    {'data_element': 'Medical Degree',
     'element':'DEGREE_CD',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'GME Institution',
     'element':'ORG_NM_GME',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'GME Institution State',
     'element':'STATE_ID_GME',
    'null': ['',-1],
    'universe': us_gmes},
    {'data_element': 'GME Begin Date',
     'element':'BEGIN_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME End Date',
     'element':'END_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME Primary Specialty',
     'element':'PRIM_SPEC_ID',
    'null': ['',-1,1883],
    'universe':all_mes},
    {'data_element': 'GME Secondary Specialty',
     'element':'SEC_SPEC_ID',
    'null': ['',-1,1883],
    'universe': all_mes},
    {'data_element': 'GME Training Type',
     'element':'TRAIN_TYPE',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME Confirmation Status',
     'element':'CONF_STS_ID',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME Status Type',
     'element':'GME_STS_TYPE_CD',
    'null': ['',-1],
    'universe':all_mes},
    {'data_element': 'GME Year in Program',
     'element':'PROG_YEAR',
    'null': ['',-1],
    'universe':all_mes},
    {'data_element': 'GME Post Grad Year',
     'element':'PROG_GRAD_YR',
    'null': ['',-1],
    'universe':all_mes}
]
dict_list = []
for typo in types:
    total = len(ALL[ALL.type==typo])
    for element in elements:
        universe = len(ALL[(ALL.type==typo)&(ALL.ME.isin(element['universe']))])
        complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        true_complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isin(element['null']))&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness'
        }
        dict_list.append(new_dict)

In [ ]:
pd.DataFrame(dict_list).to_csv(f'../../Data/Measurement/Education_Completeness_{str(datetime.date.today())}.csv', index=False)

In [ ]:
datetime.datetime.strptime(today, '%Y-%m-%d')

In [ ]:
import math
def coin_reward(minutes):
    x = 4+ 0.2*minutes + 5*(minutes-30)/30
    return math.floor(x)

In [114]:
pd.Date()

AttributeError: module 'pandas' has no attribute 'Date'

In [118]:
post_cd = post_cd[post_cd.POST_CD_ID.isin(party_addr.POST_CD_ID)]

In [120]:
post_cd[post_cd.ADDR_1.isna()]

,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE_ID,POST_CD,POST_CD_PLUS_4,LATITUDE,LONGITUDE
2775715,30793588,None,12TH EVAC HOSPITAL,None,APO 96353 00000,-1.0,00000,None,None,None
2814850,30852471,None,"House Number 224, Street Number 12",None,RAWALPINDI,-1.0,46000,None,None,None
5606158,34899576,None,Apt 14-34,None,SINGAPORE,-1.0,37208,2,None,None
5606509,34899993,None,Rua Latino Coelho 11,None,LISBON,-1.0,10501,32,None,None
5622798,34940895,None,Apt 631,None,POTOMAC,248.0,20852,None,None,None
5623841,34942128,None,Suite 1A,None,NY,222.0,10075,None,None,None
5626858,34945710,None,Apt 203,None,WASHINGTON DC,228.0,20036,None,None,None
5627915,34946967,None,Apt 2501,None,TORONTO,-1.0,5R3M9,None,None,None
5628320,34947458,None,Suite 202,None,POTOMAC,248.0,20016,None,None,None


In [135]:
post_cd = pd.merge(post_cd, state_tbl, on='STATE_ID')

In [128]:
cols = ['LOCALE NAME','PHYSICAL CITY', 'PHYSICAL STATE', 'PHYSICAL ZIP', 'PHYSICAL ZIP 4']

In [129]:
# x = pd.read_excel('../../Data/Measurement/Zip_Locale_Detail.xls', sheet_name='ZIP_DETAIL')
# y = pd.read_excel('../../Data/Measurement/Zip_Locale_Detail.xls', sheet_name='Unique_ZIP_DETAIL')
# z = pd.read_excel('../../Data/Measurement/Zip_Locale_Detail.xls', sheet_name='Other_ZIP_DETAIL')
zips = pd.concat([x[cols],y[cols],z[cols]])

In [137]:
zips.head()

,LOCALE NAME,PHYSICAL CITY,PHYSICAL STATE,PHYSICAL ZIP,PHYSICAL ZIP 4
0,ADJUNTAS,ADJUNTAS,PR,601.0,9998.0
1,AGUADA,AGUADA,PR,602.0,9998.0
2,AGUADILLA,AGUADILLA,PR,603.0,9998.0
3,RAMEY,AGUADILLA,PR,603.0,9996.0
4,AGUADILLA,AGUADILLA,PR,603.0,9998.0


In [133]:
pd.merge(post_cd, zips, left_on=['CITY', 'SRC_STATE_CD'], right_on=['PHYSICAL_ZIP','PHYSICAL ZIP 4'])

3645